# 01 - 特征提取

本notebook介绍电池数据的特征提取方法，包括：

1. **统计特征** (16维) - 协议无关的电压/电流统计量
2. **时序特征** (3通道) - 电压、电流、时间序列
3. **热力图特征** (H×W×3) - 多循环堆叠的视觉特征

## 核心更新

**16维协议无关特征**（适用于CC、CC-CV、多阶段等所有充电协议）：
- 电压统计：v_mean, v_std, v_max, v_min, v_range, v_slope
- 电流统计：i_mean, i_std, i_max, i_min, i_range, i_slope
- 充电特性：charge_time, q_total, dv_dt_mean, di_dt_mean

**3通道数据格式**：
- voltage: 电压 (V)
- current: 电流 (A)
- time: 时间 (s)

**前置要求**: 运行 `00_data_preprocessing.ipynb` 完成数据预处理

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']  # Windows 系统推荐使用 SimHei
matplotlib.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
from pathlib import Path

from src.data import (
    load_processed_batteries,
    # 统计特征
    extract_features_from_cycle,
    extract_all_features_from_battery,
    FEATURE_NAMES,
    describe_features,
    # 时序特征
    create_sequence,
    create_heatmap,
    # IC/DC物理特征
    compute_ic_curve,
    compute_dc_curve,
    find_ic_peaks,
    extract_ic_dc_features,
    IC_DC_FEATURE_NAMES,
    # 历史循环特征（跨域通用）
    HISTORICAL_CHANNELS,
    create_cycle_sequence,
    create_historical_summary_features,
    compute_degradation_trend_features,
)

# 加载数据
batteries = load_processed_batteries('../data/processed/MATR')
print(f"加载了 {len(batteries)} 个电池")

battery = batteries[0]
print(f"示例电池: {battery.cell_id}, {len(battery)} 循环")

## 1. 统计特征 (16维) - 协议无关

从完整充电曲线提取的统计量（适用于所有充电协议）：

| 编号 | 特征名 | 说明 |
|------|--------|------|
| 1-4 | v_mean, v_std, v_max, v_min | 电压统计量 |
| 5-6 | v_range, v_slope | 电压范围和变化率 |
| 7-10 | i_mean, i_std, i_max, i_min | 电流统计量 |
| 11-12 | i_range, i_slope | 电流范围和变化率 |
| 13-14 | charge_time, q_total | 充电时间和总电量 |
| 15-16 | dv_dt_mean, di_dt_mean | 电压/电流变化率 |

In [ ]:
# 提取单循环特征
cycle = battery.cycles[100]
features = extract_features_from_cycle(cycle)

print("统计特征 (16维):")
print(f"形状: {features.shape}")
print(f"\n特征名称和值:")
for name, val in zip(FEATURE_NAMES, features):
    print(f"  {name}: {val:.6f}")

In [ ]:
# 提取所有循环的特征
all_features = extract_all_features_from_battery(battery)
print(f"所有循环特征形状: {all_features.shape}  # (循环数, 16)")

# 可视化特征随循环的变化
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

features_to_plot = ['v_mean', 'i_mean', 'charge_time', 'v_slope']
for ax, feat_name in zip(axes.flat, features_to_plot):
    idx = FEATURE_NAMES.index(feat_name)
    ax.plot(all_features[:, idx])
    ax.set_xlabel('循环数')
    ax.set_ylabel(feat_name)
    ax.set_title(f'{feat_name} 随循环变化')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. IC/DC物理特征 (12维)

基于电化学物理意义的特征：
- **IC曲线** (dQ/dV): 增量容量曲线，峰值反映电极材料相变
- **DC曲线** (dV/dQ): 微分电压曲线，IC曲线的倒数

这些特征与充电协议无关，具有更好的跨域泛化性。

In [ ]:
# 计算IC曲线
cycle = battery.cycles[10]
v_ic, ic = compute_ic_curve(cycle.voltage, cycle.current, cycle.time)

# 计算DC曲线
q_dc, dc = compute_dc_curve(cycle.voltage, cycle.current, cycle.time)

# 找IC峰
peaks = find_ic_peaks(v_ic, ic, min_height=0.1)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# IC曲线
ax = axes[0]
ax.plot(v_ic, ic, 'b-', linewidth=1)
for peak in peaks[:2]:
    ax.axvline(x=peak['position'], color='r', linestyle='--', alpha=0.5)
    ax.annotate(f"Peak: {peak['position']:.3f}V\nHeight: {peak['height']:.2f}",
                xy=(peak['position'], peak['height']),
                xytext=(10, 10), textcoords='offset points')
ax.set_xlabel('电压 (V)')
ax.set_ylabel('dQ/dV (Ah/V)')
ax.set_title('IC曲线 (增量容量)')
ax.grid(True, alpha=0.3)

# DC曲线
ax = axes[1]
ax.plot(q_dc, dc, 'g-', linewidth=1)
ax.set_xlabel('容量 (Ah)')
ax.set_ylabel('dV/dQ (V/Ah)')
ax.set_title('DC曲线 (微分电压)')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 提取IC/DC物理特征
ic_dc_features = extract_ic_dc_features(cycle)

print("IC/DC物理特征 (12维):")
for name, val in zip(IC_DC_FEATURE_NAMES, ic_dc_features):
    print(f"  {name}: {val:.6f}")

In [ ]:
# IC曲线随循环的演变
cycles_to_compare = [10, len(battery)//3, 2*len(battery)//3, len(battery)-10]

fig, ax = plt.subplots(figsize=(10, 6))
colors = plt.cm.coolwarm(np.linspace(0, 1, len(cycles_to_compare)))

for idx, c_idx in enumerate(cycles_to_compare):
    if c_idx < len(battery):
        cycle = battery.cycles[c_idx]
        v_ic, ic = compute_ic_curve(cycle.voltage, cycle.current, cycle.time)
        ax.plot(v_ic, ic, color=colors[idx], linewidth=1.5,
                label=f'Cycle {cycle.cycle_number} (SOH={cycle.soh:.2%})')

ax.set_xlabel('电压 (V)')
ax.set_ylabel('dQ/dV (Ah/V)')
ax.set_title('IC曲线随循环的演变')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. 时序特征

保留完整时序信息的特征表示。

In [ ]:
# 单循环时序
cycle = battery.cycles[100]
seq = create_sequence(cycle, num_samples=200)

print(f"单循环时序形状: {seq.shape}  # (采样点, 3通道)")

# 可视化
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
channel_names = ['电压', '电流', '时间']

for ax, (i, name) in zip(axes.flat, enumerate(channel_names)):
    ax.plot(seq[:, i])
    ax.set_xlabel('采样点')
    ax.set_ylabel(name)
    ax.set_title(f'{name}序列')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 多循环热力图
heatmap = create_heatmap(battery, window_size=100, num_samples=200)
print(f"热力图形状: {heatmap.shape}  # (循环数, 采样点, 3通道)")

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
titles = ['电压热力图', '电流热力图', '时间热力图']

for ax, (i, title) in zip(axes, enumerate(titles)):
    im = ax.imshow(heatmap[:, :, i], aspect='auto', cmap='viridis')
    ax.set_xlabel('采样点')
    ax.set_ylabel('循环')
    ax.set_title(title)
    plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

## 4. 历史循环特征（跨域通用）

专为跨数据集场景设计的特征：
- **3通道**: 电压、电流、时间
- **历史摘要**: 从全部历史中均匀采样
- **退化趋势**: 8维辅助特征

In [ ]:
print(f"历史循环通道: {HISTORICAL_CHANNELS}")

# 单循环3通道特征
cycle = battery.cycles[100]
seq = create_cycle_sequence(cycle, num_samples=200)
print(f"单循环3通道形状: {seq.shape}")

# 历史摘要特征
summary = create_historical_summary_features(
    battery,
    end_cycle_idx=200,  # 使用前200个循环
    num_samples=200,
    num_summary_cycles=50,  # 采样50个循环
)
print(f"历史摘要形状: {summary.shape}  # (采样循环数, 采样点, 3通道)")

# 退化趋势特征
trend = compute_degradation_trend_features(battery, end_cycle_idx=200)
print(f"退化趋势特征: {trend.shape}")
print(f"  当前SOH: {trend[0]:.4f}")
print(f"  退化斜率: {trend[1]:.6f}")
print(f"  容量损失率: {trend[4]:.4f}")

In [ ]:
# 可视化历史摘要热力图
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
titles = ['电压', '电流', '时间']

for ax, (i, title) in zip(axes, enumerate(titles)):
    im = ax.imshow(summary[:, :, i], aspect='auto', cmap='viridis')
    ax.set_xlabel('采样点')
    ax.set_ylabel('历史循环索引')
    ax.set_title(f'{title}通道 (归一化)')
    plt.colorbar(im, ax=ax)

plt.suptitle(f'历史循环摘要 - 使用前200个循环', fontsize=12)
plt.tight_layout()
plt.show()

## 5. 特征选择建议

| 场景 | 推荐特征 | 说明 |
|------|----------|------|
| 单数据集，快速实验 | 统计特征(16维) | 简单高效 |
| 需要物理解释 | IC/DC特征(12维) | 有电化学意义 |
| 深度学习模型 | 时序特征 | 保留完整信息 |
| 跨数据集预测 | 历史循环特征(3通道) |  |
| SOH+RUL联合预测 | 历史摘要+退化趋势 | 利用完整历史 |

## 下一步

特征提取完成后，进入模型训练：
- `02_model_training.ipynb` - 模型训练与SOH/RUL预测